In [250]:
# Hermawan - NEVTIK
import requests
import pandas
import csv

In [251]:
shopee_url = "https://shopee.co.id/"
keyword_search = "Laptop"
limit = 100  # Max 100 (bug)
official = 1 # Change value to 1 for filterring official store

if official == 1:
    official_shop = "ShopeeMall"
else:
    official_shop = "Shopee"

csv_file = open('{}_{}_{}.csv'.format(official_shop,keyword_search,limit),'w')
csv_writer = csv.writer(csv_file)

In [252]:
headers = {
    #Change 'User-Agent' equivalent to your browser
    'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",
    'referrer' : '{}search?keyword={}'.format(shopee_url,keyword_search)
}

In [253]:
product_list,sold_list,price_list,brand_list,rating,stock,location,price_before_discount = [],[],[],[],[],[],[],[]
columns = ["Product","Brand","Location","Sold","Stock","Rating","Price_Before_Discount","Price"]

class GetID:
    def __init__(self,search,limit):
        API_id_url = "https://shopee.co.id/api/v2/search_items/?by=relevancy&keyword={}&limit={}&newest=0&official_mall={}&order=desc&page_type=search&version=2".format(search,limit,official)
        self.api_requests = requests.get(API_id_url,headers = headers).json()
        self.shop_id,self.item_id = [],[]
        for item in self.api_requests["items"]:
            self.item_id.append(item["itemid"])
            self.shop_id.append(item["shopid"])
            
get_item_id = GetID(keyword_search,limit).item_id
get_shop_id = GetID(keyword_search,limit).shop_id

def get_shopee():
    for index in range(len(get_item_id)):
        API_item_url = "https://shopee.co.id/api/v2/item/get?itemid={}&shopid={}".format(get_item_id[index],get_shop_id[index])
        api_requests = requests.get(API_item_url, headers=headers).json()
        yield api_requests

for i in get_shopee():
    item = i['item']
    product_list.append(item['name'])
    sold_list.append(item['historical_sold'])
    price_list.append(int(str(item['price_min'])[:-5]))
    location.append(item['shop_location'])
    rating.append(round(float(item['item_rating']['rating_star']),1))
    stock.append(item['stock'])
    brand_list.append(item['brand'])
    desc_list.append(item['description'])
    try: 
        price_before_discount.append(int(str(item['price_before_discount'])[:-5]))
    except: 
        price_before_discount.append(int(str(item['price_min'])[:-5]))

In [254]:
product_detail = {"Product":product_list,"Brand":brand_list,"Location":location,"Sold":sold_list,"Stock":stock,"Rating":rating,"Price_Before_Discount":price_before_discount,"Price":price_list}
csv_writer.writerow(columns)

for item_to_csv in range(len(product_list)):
    csv_writer.writerow([product_detail["Product"][item_to_csv],
                         product_detail["Brand"][item_to_csv],
                         product_detail["Location"][item_to_csv],
                         product_detail["Sold"][item_to_csv],
                         product_detail["Stock"][item_to_csv],
                         product_detail["Rating"][item_to_csv],
                         product_detail["Price_Before_Discount"][item_to_csv],
                         product_detail["Price"][item_to_csv]
                        ])
    
csv_file.close()
dataframe = pandas.DataFrame(product_detail,columns=columns)
dataframe

,Product,Brand,Location,Sold,Stock,Rating,Price_Before_Discount,Price
0,Meja Laptop Lipat Portable Stand Aluminium Coo...,GOTO,KOTA TANGERANG,1311,198,4.8,381250,152500
1,Promate Laptop Softcase Sleeve - Portfolio-M T...,0,KOTA JAKARTA SELATAN,0,9899,0.0,199000,189000
2,HP Laptop 14s-cf0063TU/i3-7020U/4GB RAM/1TB HD...,0,KOTA JAKARTA UTARA,52,4,4.6,5849000,5791000
3,"Lenovo Laptop IP330 14AST 14"" / AMD A9-9425 / ...",Lenovo,KOTA JAKARTA UTARA,88,3,4.6,4499000,4251000
4,Laptop Gaming Acer Nitro 5 AN515-54 ( NH.Q59SN...,Acer,KOTA TANGERANG SELATAN,0,1,0.0,14299000,12999000
...,...,...,...,...,...,...,...,...
95,"LENOVO IPL340-15IRH - SLID, [i7-9750H/16GB/512...",Lenovo,KOTA MEDAN,0,3,0.0,14660000,14660000
96,Asus X441Ub-Ga322T I3-7020U/4Gb/1Tb/Mx110/14.0...,0,KOTA JAKARTA UTARA,1,1,5.0,6699000,5792000
97,Laptop Acer Spin 5 SP513-52N NX.GR7SN.004 - FR...,Acer,KOTA TANGERANG SELATAN,0,2,0.0,12499000,11999000
98,Laptop Gaming Lenovo Legion Y540-AKID,Lenovo Legion,KOTA TANGERANG SELATAN,0,2,0.0,14499000,14354010
